# AES

# Recherche de la clé dans les images

In [21]:
from PIL import Image

def dechiffrer_image(path):
    image = Image.open(path)
    pixels = image.load()
    (largeur, hauteur) = image.size
    return ''.join([str(pixels[x, y] % 2) for y in range(hauteur) for x in range(largeur)])[:64]

key = dechiffrer_image('data/rossignol2.bmp')
print(f'La clé est en binaire : {key} (en hexadécimal : {hex(int(key, 2))})')

La clé est en binaire : 1110011101101101001100010011111110010010101110011001000001001100 (en hexadécimal : 0xe76d313f92b9904c)
